In [49]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [50]:
!pip install torchtext==0.10.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [51]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [52]:
import torch
from torchtext.legacy import datasets
from torchtext.legacy import data


SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL =data.LabelField(dtype = torch.float)

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [53]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

We can see how many examples are in each split by checking their length.

In [54]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))
type(train_data)

Number of training examples: 25000
Number of testing examples: 25000
{'text': ['I', 'have', 'seen', 'Slaughter', 'High', 'several', 'times', 'over', 'the', 'years', ',', 'and', 'always', 'found', 'it', 'was', 'an', 'enjoyable', 'slasher', 'flick', 'with', 'an', 'odd', 'sense', 'of', 'humor', ',', 'but', 'I', 'never', 'knew', 'that', 'it', 'was', 'filmed', 'in', 'the', 'UK', ',', 'and', 'I', 'never', 'knew', 'that', 'the', 'actor', 'that', 'plays', 'Marty', 'Rantzen', '(', 'Simon', 'Scuddamore', ')', 'committed', 'suicide', 'after', 'the', 'film', 'was', 'released', '.', 'I', 'guess', 'I', 'did', 'notice', 'while', 'watching', 'it', 'last', 'night', 'that', 'the', 'actors', 'phrasing', 'seems', 'rather', 'odd', 'for', 'Americans', ',', 'and', 'a', 'few', 'of', 'them', 'are', "n't", 'very', 'good', 'at', 'hiding', 'their', 'English', 'accents.<br', '/><br', '/>All', 'that', 'aside', 'though', ',', 'this', 'is', 'the', 'tale', 'of', 'the', 'class', 'nerd', ',', 'Marty', ',', 'who', 'is', 

torchtext.legacy.datasets.imdb.IMDB

In [55]:
import pandas as pd
import json

In [57]:
def load_data(path, type):

  df = []
  print(path)
  with open(path, encoding="utf-8") as f:
    for i in f:
      i = json.loads(i)
      tweet = i['postText']
      article_title = i['targetTitle']
      article = ' '.join(i['targetParagraphs'])
      label = i['tags']

      tweet = tweet[0]
      label = label[0]

      final_label = 0

      if type == "1vs1" and label != 'multi':
        final_label = (label == 'phrase' or label == 'phrases') 
      elif type == "012":
        if label == 'passage':
          final_label = 1
        elif label == 'mutli':
          final_label = 2
      else:
        continue

      df += [
          {
              'text': tweet + " - " + article_title + article,
              'label': final_label
          }
      ]

    return pd.DataFrame(df)

In [58]:
from torch.utils.data import Dataset, DataLoader
class CustomTextDataset(Dataset):
    def __init__(self, text, labels):
        self.labels = labels
        self.text = text
def __len__(self):
        return len(self.labels)
def __getitem__(self, index):
        label = self.labels[index]
        text = self.text[index]
        sample = {"Text": text, "Class": label}
        return sample

In [59]:
train_dataset_path = "/content/train.jsonl"
test_dataset_path = "/content/test.jsonl"
validation_dataset_path = "/content/validation.jsonl"

class_type = "1vs1"

train_data = load_data(train_dataset_path, class_type)
train_data

/content/train.jsonl


,text,label
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole...,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [60]:
# import random

# train_data, valid_data = train_data.split(random_state = random.seed(SEED))

valid_data = load_data(validation_dataset_path, class_type)
print(type(valid_data))

test_data = load_data(validation_dataset_path, class_type)


/content/validation.jsonl
<class 'pandas.core.frame.DataFrame'>
/content/validation.jsonl


In [61]:
test_data

,text,label
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
652,"Dog Dies One Hour After Hiking With His Owner,...",False
653,This is what happens when you leave a hotel cl...,False
654,This Texas GOP elector announces that he won't...,True
655,WikiLeaks' Julian Assange Reported Dead - Wiki...,False


In [73]:
train_data

,text,label
0,"Wes Welker Wanted Dinner With Tom Brady, But P...",False
1,NASA sets date for full recovery of ozone hole...,True
2,This is what makes employees happy -- and it's...,True
3,The perfect way to cook rice so that it's perf...,True
4,What happens if your new AirPods get lost or s...,False
...,...,...
2636,If You See A Purple Butterfly Sticker At The H...,False
2637,Has Facebook's video explosion completely shak...,False
2638,Cop Is Eating At A Chili's When Teen Hands Him...,False
2639,You need to see this Twitter account that pred...,True


In [74]:
valid_data

,text,label
0,Five Nights at Freddy’s Sequel Delayed for Wei...,False
1,Here’s how much you should be tipping your hai...,True
2,A man swallowed a microSD card and you won't b...,False
3,This popular soda could cure your hangovers sc...,True
4,The anytime snack you won't feel guilty about ...,True
...,...,...
652,"Dog Dies One Hour After Hiking With His Owner,...",False
653,This is what happens when you leave a hotel cl...,False
654,This Texas GOP elector announces that he won't...,True
655,WikiLeaks' Julian Assange Reported Dead - Wiki...,False


In [75]:
import pandas as pd
from torchtext.legacy.data import Field, Example, Dataset

def example_cmp(ex1, ex2):
    return len(ex1.text) < len(ex2.text)

def get_tt_from_df(df):
  examples = []
  for i, row in df.iterrows():
      text = row["text"]
      label = row["label"]
      example = Example.fromlist([text, label], fields=[("text", TEXT), ("label", LABEL)])
      examples.append(example)

  # Create a dataset from the list of examples
  dataset = Dataset(examples, fields=[("text", TEXT), ("label", LABEL)])
  return dataset

In [76]:
train_data = get_tt_from_df(train_data)
valid_data = get_tt_from_df(valid_data)
# test_data = get_tt_from_df(test_data)
print(type(train_data))
print(type(valid_data))
# print(type(test_data))

<class 'torchtext.legacy.data.dataset.Dataset'>
<class 'torchtext.legacy.data.dataset.Dataset'>


In [77]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "fasttext.simple.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [78]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [79]:
print(TEXT.vocab.freqs.most_common(20))

[(',', 68596), ('.', 57321), ('the', 56687), ('to', 34869), ('a', 29467), ('of', 28762), ('and', 26993), ('"', 25274), ('in', 20367), ('that', 16007), ('-', 15652), ('is', 13194), ('for', 10974), ('it', 10595), ('on', 9740), ("'s", 9041), ('The', 8820), ('I', 8347), ('you', 8197), ('with', 8017)]


In [80]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', ',', '.', 'the', 'to', 'a', 'of', 'and', '"']


In [81]:
BATCH_SIZE = 4

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

We are working with  cuda


## Bidirectional Multi-Layer LSTM




In [82]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        #TO-DO
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, pad_idx)

        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(
            input_size = embedding_dim, 
            hidden_size = hidden_dim,
            num_layers = n_layers,
            dropout = dropout,
            bidirectional = bidirectional
            )

        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(in_features=hidden_dim * 2 , out_features=output_dim)

        #4. Initialize Dropout
        self.dropout_layer = nn.Dropout(p=dropout)
        
    def forward(self, text, text_lengths):
        #text = [sent_len, batch_size]

        

        embeddings = self.embedding(text)
        dropout = self.dropout_layer(embeddings)
        output, (hidden, c0) = self.lstm(dropout)
        hidden = self.dropout_layer(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
            
        return self.fc(hidden)

In [83]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [84]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,220,857 trainable parameters


In [85]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 300])


In [86]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [ 0.1661,  0.3699,  0.3510,  ..., -0.3465, -0.3897, -0.2221],
        [ 0.2670,  0.5171,  0.0501,  ..., -0.3363,  0.0985, -0.1113],
        [ 0.0438,  0.3707,  0.1683,  ..., -0.2434,  0.0574, -0.0324]])

In [87]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2013,  0.0104,  0.1623,  ..., -0.0931, -0.1408, -0.1326],
        ...,
        [ 0.1661,  0.3699,  0.3510,  ..., -0.3465, -0.3897, -0.2221],
        [ 0.2670,  0.5171,  0.0501,  ..., -0.3363,  0.0985, -0.1113],
        [ 0.0438,  0.3707,  0.1683,  ..., -0.2434,  0.0574, -0.0324]])


In [88]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [89]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [90]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [91]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [92]:
from sklearn.metrics import confusion_matrix

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    final_predictions = None

    ttp, ttn, tfp, tfn = 0,0,0,0
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            final_predictions = predictions
            rounded_preds = torch.round(torch.sigmoid(final_predictions))

            final_predictions = rounded_preds.cpu().detach().numpy()
            true_labels = batch.label.cpu().detach().numpy()


            # calculate confusion matrix
            confusion = confusion_matrix(true_labels, final_predictions).ravel()

            # unpack TP, TN, FP, FN
            if len(confusion) == 4:
              tn, fp, fn, tp = confusion
              ttp += tp
              ttn += tn
              tfp += fp
              tfn += fn
          



            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()


    print(ttp, ttn, tfp, tfn)
    print((ttp + ttn) / (ttp+ttn+tfp+tfn))
    return epoch_loss / len(iterator), epoch_acc / len(iterator), final_predictions 

In [93]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [94]:
N_EPOCHS = 5
path = F"/content/gdrive/My Drive/bilstm_model.pt"
best_valid_loss = float('inf')

from sklearn.metrics import accuracy_score
final_predictions = None

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc, final_predictions = evaluate(model, valid_iterator, criterion)

    model.train()
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

145 227 100 172
0.577639751552795
Epoch: 01 | Epoch Time: 1m 20s
	Train Loss: 0.670 | Train Acc: 58.70%
	 Val. Loss: 0.672 |  Val. Acc: 58.79%
190 205 122 123
0.6171875
Epoch: 02 | Epoch Time: 1m 13s
	Train Loss: 0.604 | Train Acc: 67.40%
	 Val. Loss: 0.654 |  Val. Acc: 62.88%
159 237 86 150
0.6265822784810127
Epoch: 03 | Epoch Time: 1m 13s
	Train Loss: 0.491 | Train Acc: 76.82%
	 Val. Loss: 0.794 |  Val. Acc: 64.24%
166 237 94 147
0.6257763975155279
Epoch: 04 | Epoch Time: 1m 12s
	Train Loss: 0.363 | Train Acc: 84.38%
	 Val. Loss: 0.904 |  Val. Acc: 63.48%
199 188 139 110
0.6084905660377359
Epoch: 05 | Epoch Time: 1m 13s
	Train Loss: 0.276 | Train Acc: 88.39%
	 Val. Loss: 1.014 |  Val. Acc: 62.27%


In [95]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc, pred = evaluate(model, valid_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%', pred)

190 205 122 123
0.6171875
Test Loss: 0.654 | Test Acc: 62.88% [1.]
